In [2]:
from keras.layers import Conv2D, Conv2DTranspose, Dense, Flatten, Reshape
from keras.models import Sequential, Model
from keras.utils.vis_utils import plot_model
from keras import losses
import numpy as np
import tensorflow as tf
import prepare_data_for_clustering_utils as prep

In [3]:
x = np.load('../npy_files/features_All16.npy')
x.shape


(29808, 16, 5)

In [171]:
X_train = x[:,:16,:4]

In [195]:
X_train.shape

(1878, 16, 4)

In [200]:
# X_train = x.copy()
X_train = X_train.reshape(X_train.shape[0], 4, 4, 4)

In [196]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)

### Autoencoder Base

In [56]:
latent_dim = 4 
class Autoencoder_Base(Model):
  def __init__(self, latent_dim):
    super(Autoencoder_Base, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      Flatten(),
      Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      Dense(125, activation='sigmoid'),
      Reshape((25, 5))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder_Base(latent_dim)

In [57]:
autoencoder.compile(optimizer='adam', loss='mae')
autoencoder.build(input_shape=(1,25,5))

In [58]:
autoencoder.encoder.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (1, 125)                  0         
                                                                 
 dense_28 (Dense)            (1, 4)                    504       
                                                                 
Total params: 504
Trainable params: 504
Non-trainable params: 0
_________________________________________________________________


In [59]:
autoencoder.fit(x, x,
                epochs=5,
                shuffle=True)

Epoch 1/5
59/59 [==============================] - 0s 2ms/step - loss: 41.6211
Epoch 2/5
59/59 [==============================] - 0s 965us/step - loss: 41.4965
Epoch 3/5
59/59 [==============================] - 0s 969us/step - loss: 41.4430
Epoch 4/5
59/59 [==============================] - 0s 1ms/step - loss: 41.4271
Epoch 5/5
59/59 [==============================] - 0s 1ms/step - loss: 41.4222


### Autoencoder Dense

In [4]:
import tensorflow as tf

class DtwLoss(tf.keras.losses.Loss):
    def __init__(self, batch_size: int = 32):
        super(DtwLoss, self).__init__()
        self.batch_size = batch_size

    def call(self, y_true, y_pred):
        tmp = []
        for item in range(self.batch_size):
            tf.print(f'Working on batch: {item}\n')
            s = y_true[item, :]
            t = y_pred[item, :]
            n, m = len(s), len(t)
            dtw_matrix = []
            for i in range(n + 1):
                line = []
                for j in range(m + 1):
                    if i == 0 and j == 0:
                        line.append(0)
                    else:
                        line.append(np.inf)
                dtw_matrix.append(line)

            for i in range(1, n + 1):
                for j in range(1, m + 1):
                    cost = tf.abs(s[i - 1] - t[j - 1])
                    last_min = tf.reduce_min([dtw_matrix[i - 1][j], dtw_matrix[i][j - 1], dtw_matrix[i - 1][j - 1]])
                    dtw_matrix[i][j] = tf.cast(cost, dtype=tf.float32) + tf.cast(last_min, dtype=tf.float32)

            temp = []
            for i in range(len(dtw_matrix)):
                temp.append(tf.stack(dtw_matrix[i]))

            tmp.append(tf.stack(temp)[n, m])
        return tf.reduce_mean(tmp)

In [11]:
x.shape

(29808, 16, 5)

In [16]:
class Autoencoder_Dense(Model):
  def __init__(self, latent_dim):
    super(Autoencoder_Dense, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = Sequential([
      Flatten(),
      Dense(32, activation="relu",name='dense1'),
      Dense(16, activation="relu",name='dense2'),
      Dense(latent_dim, activation="relu",name='dense3')])

    self.decoder = Sequential([
      Dense(6, activation="relu",name='dedense1'),
      Dense(10, activation="relu",name='dedense2'),
      Dense(16, activation="sigmoid",name='dedense3'),
      Reshape((16, 1) , name='reshape1')
      ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder_Dense(4)

In [17]:
autoencoder.compile(optimizer='adam', loss=DtwLoss(batch_size=1))
autoencoder.build(input_shape=(1,16,1))

In [18]:
autoencoder.encoder.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (1, 16)                   0         
                                                                 
 dense1 (Dense)              (1, 32)                   544       
                                                                 
 dense2 (Dense)              (1, 16)                   528       
                                                                 
 dense3 (Dense)              (1, 4)                    68        
                                                                 
Total params: 1,140
Trainable params: 1,140
Non-trainable params: 0
_________________________________________________________________


In [19]:
autoencoder.decoder.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dedense1 (Dense)            (1, 6)                    30        
                                                                 
 dedense2 (Dense)            (1, 10)                   70        
                                                                 
 dedense3 (Dense)            (1, 16)                   176       
                                                                 
 reshape1 (Reshape)          (1, 16, 1)                0         
                                                                 
Total params: 276
Trainable params: 276
Non-trainable params: 0
_________________________________________________________________


In [23]:
autoencoder.fit(x[:,:,:1], x[:,:,:1],
                epochs=5,
                shuffle=True)

Epoch 1/5


ValueError: in user code:

    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\asafavra\AppData\Local\Temp\ipykernel_22596\261076392.py", line 28, in call  *
        last_min = tf.reduce_min([dtw_matrix[i - 1][j], dtw_matrix[i][j - 1], dtw_matrix[i - 1][j - 1]])

    ValueError: Shapes must be equal rank, but are 1 and 0
    	From merging shape 1 with other shapes. for '{{node DtwLoss/Rank_1/packed}} = Pack[N=3, T=DT_FLOAT, axis=0](DtwLoss/Rank_1/packed/0, DtwLoss/add, DtwLoss/Rank_1/packed/2)' with input shapes: [], [1], [].


### Autoencoder conv2d

In [201]:
def CAE_mod(input_shape=(4,4,4)):
    model = Sequential()
    if input_shape[0] % 6 == 0:
        pad3 = 'same'
    else:
        pad3 = 'valid'
    model.add(Conv2D(16, 5, strides=2, padding='same', activation='relu', name='conv1', input_shape=input_shape))

    model.add(Flatten())
    model.add(Dense(units=4, name='embedding'))
    model.add(Dense(units=4*4*4, activation='relu'))

    model.add(Reshape((2,2,16)))

    model.add(Conv2DTranspose(4, 5, strides=2, padding='same', activation='sigmoid', name='deconv1'))

    model.summary()
    return model


In [202]:
cae = CAE_mod()

Model: "sequential_69"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 2, 2, 16)          1616      
                                                                 
 flatten_42 (Flatten)        (None, 64)                0         
                                                                 
 embedding (Dense)           (None, 4)                 260       
                                                                 
 dense_68 (Dense)            (None, 64)                320       
                                                                 
 reshape_38 (Reshape)        (None, 2, 2, 16)          0         
                                                                 
 deconv1 (Conv2DTranspose)   (None, 4, 4, 4)           1604      
                                                                 
Total params: 3,800
Trainable params: 3,800
Non-train

In [208]:
cae.compile(optimizer='adam', loss='mse')
history = cae.fit(X_train, X_train, batch_size=128, epochs=200,verbose = 0)

Epoch 1/200
15/15 - 1s - loss: 0.7087 - 511ms/epoch - 34ms/step
Epoch 2/200
15/15 - 0s - loss: 0.7084 - 30ms/epoch - 2ms/step
Epoch 3/200
15/15 - 0s - loss: 0.7082 - 35ms/epoch - 2ms/step
Epoch 4/200
15/15 - 0s - loss: 0.7081 - 37ms/epoch - 2ms/step
Epoch 5/200
15/15 - 0s - loss: 0.7079 - 34ms/epoch - 2ms/step
Epoch 6/200
15/15 - 0s - loss: 0.7079 - 36ms/epoch - 2ms/step
Epoch 7/200
15/15 - 0s - loss: 0.7078 - 35ms/epoch - 2ms/step
Epoch 8/200
15/15 - 0s - loss: 0.7079 - 32ms/epoch - 2ms/step
Epoch 9/200
15/15 - 0s - loss: 0.7080 - 36ms/epoch - 2ms/step
Epoch 10/200
15/15 - 0s - loss: 0.7079 - 35ms/epoch - 2ms/step
Epoch 11/200
15/15 - 0s - loss: 0.7079 - 38ms/epoch - 3ms/step
Epoch 12/200
15/15 - 0s - loss: 0.7080 - 34ms/epoch - 2ms/step
Epoch 13/200
15/15 - 0s - loss: 0.7080 - 35ms/epoch - 2ms/step
Epoch 14/200
15/15 - 0s - loss: 0.7081 - 36ms/epoch - 2ms/step
Epoch 15/200
15/15 - 0s - loss: 0.7081 - 36ms/epoch - 2ms/step
Epoch 16/200
15/15 - 0s - loss: 0.7082 - 38ms/epoch - 3ms/step

### Autoencoder LSTM

In [99]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

model = Sequential()
model.add(LSTM(5, activation='relu', input_shape=(25,5), name= 'embedding'))
model.add(RepeatVector(25))
model.add(LSTM(5, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(5)))
model.compile(optimizer='adam', loss='mse')

In [100]:
model.summary()

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (LSTM)            (None, 5)                 220       
                                                                 
 repeat_vector_6 (RepeatVect  (None, 25, 5)            0         
 or)                                                             
                                                                 
 lstm_8 (LSTM)               (None, 25, 5)             220       
                                                                 
 time_distributed_6 (TimeDis  (None, 25, 5)            30        
 tributed)                                                       
                                                                 
Total params: 470
Trainable params: 470
Non-trainable params: 0
_________________________________________________________________


In [101]:
hidden = model.get_layer(name='embedding').output


In [102]:
hidden

<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'embedding')>

In [95]:
model.fit(X_train, X_train, epochs=200, verbose=1)

Epoch 1/200
59/59 [==============================] - 2s 13ms/step - loss: 1.0172
Epoch 2/200
59/59 [==============================] - 1s 16ms/step - loss: 0.9955
Epoch 3/200
59/59 [==============================] - 1s 13ms/step - loss: 0.9792
Epoch 4/200
59/59 [==============================] - 1s 13ms/step - loss: 0.9586
Epoch 5/200
59/59 [==============================] - 1s 13ms/step - loss: 0.9425
Epoch 6/200
59/59 [==============================] - 1s 12ms/step - loss: 0.9302
Epoch 7/200
59/59 [==============================] - 1s 13ms/step - loss: 0.9221
Epoch 8/200
59/59 [==============================] - 1s 14ms/step - loss: 0.9154
Epoch 9/200
59/59 [==============================] - 1s 14ms/step - loss: 0.9102
Epoch 10/200
59/59 [==============================] - 1s 14ms/step - loss: 0.9054
Epoch 11/200
59/59 [==============================] - 1s 13ms/step - loss: 0.9015
Epoch 12/200
59/59 [==============================] - 1s 12ms/step - loss: 0.8982
Epoch 13/200
59/59 [=====

In [96]:
model.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 4)                 160       
                                                                 
 repeat_vector_5 (RepeatVect  (None, 25, 4)            0         
 or)                                                             
                                                                 
 lstm_6 (LSTM)               (None, 25, 5)             200       
                                                                 
 time_distributed_5 (TimeDis  (None, 25, 5)            30        
 tributed)                                                       
                                                                 
Total params: 390
Trainable params: 390
Non-trainable params: 0
_________________________________________________________________


### GRU

### LSTMPEEP

In [85]:
encoder = tfa.rnn.PeepholeLSTMCell(4 , activation=tf.nn.tanh,input_shape=(25,5),name="encoder")
rnn_enc = tf.keras.layers.RNN(encoder, return_sequences=True, return_state=True)

decoder = tfa.rnn.PeepholeLSTMCell(4 , activation=tf.nn.tanh,name="decoder")
rnn_dec = tf.keras.layers.RNN(decoder, return_sequences=True, return_state=True)

In [86]:
import tensorflow_addons as tfa
model = Sequential()
model.add(rnn_enc)
model.add(RepeatVector(25))
model.add(rnn_dec)
model.add(TimeDistributed(Dense(5)))
model.compile(optimizer='adam', loss='mse')

In [87]:
model.fit(x, x, epochs=10, verbose=1)

Epoch 1/10


ValueError: in user code:

    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Projects\TAU\D

In [88]:
x.shape

(1878, 25, 5)

In [ ]:
x[:,:,]

### Check DCEC